In [1]:
%load_ext autoreload

In [2]:
import os

import numpy as np

from libertem import api
from libertem.udf.sum import SumUDF
from libertem.udf.base import UDF

c:\users\weber\documents\libertem\libertem\src\libertem\utils\devices.py:13: RuntimeWarning: ImportError('CuPy is not correctly installed.\n\nIf you are using wheel distribution (cupy-cudaXX), make sure that the version of CuPy you installed matches with the version of CUDA on your host.\nAlso, confirm that only one CuPy package is installed:\n  $ pip freeze\n\nIf you are building CuPy from source, please check your environment, uninstall CuPy and reinstall it with:\n  $ pip install cupy --no-cache-dir -vvvv\n\nCheck the Installation Guide for details:\n  https://docs-cupy.chainer.org/en/latest/install.html\n\noriginal error: DLL load failed: The specified module could not be found.')
  warnings.warn(repr(e), RuntimeWarning)


In [3]:
%autoreload
ctx = api.Context()

In [4]:
ctx.executor.client.scheduler_info()

{'type': 'Scheduler',
 'id': 'Scheduler-cc13e512-ff01-4dfc-bb7e-16a21b348849',
 'address': 'tcp://192.168.178.52:58598',
 'services': {},
 'workers': {'tcp://192.168.178.52:58618': {'type': 'Worker',
   'id': 'cpu-1',
   'host': '192.168.178.52',
   'resources': {'CPU': 1, 'compute': 1},
   'local_directory': 'C:\\Users\\weber\\Documents\\LiberTEM\\LiberTEM\\prototypes\\cupy\\dask-worker-space\\worker-p0whzvdo',
   'name': 'cpu-1',
   'nthreads': 1,
   'memory_limit': 2065603584,
   'last_seen': 1589308959.182538,
   'services': {},
   'metrics': {'cpu': 0.0,
    'memory': 36118528,
    'time': 1589308954.7392683,
    'read_bytes': 0.0,
    'write_bytes': 0.0,
    'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}},
   'nanny': 'tcp://192.168.178.52:58601'},
  'tcp://192.168.178.52:58619': {'type': 'Worker',
   'id': 'service-0',
   'host': '192.168.178.52',
   'resources': {},
   'local_directory': 

In [5]:
def info():
    import os
    return (os.environ.get("LIBERTEM_USE_CPU", None), os.environ.get("LIBERTEM_USE_CUDA", None))

In [6]:
ctx.executor.client.run(info)

{'tcp://192.168.178.52:58618': ('1', None),
 'tcp://192.168.178.52:58619': (None, None),
 'tcp://192.168.178.52:58621': ('0', None)}

In [7]:
data = np.ones((4, 4, 4, 4))

ds = ctx.load("memory", data=data, num_partitions=16)

In [8]:
class DebugUDF(UDF):
    def get_result_buffers(self):
        return {
            'debug': self.buffer(kind="single", dtype="object"),
            'on_device': self.buffer(kind="sig", dtype=np.float32, where="device"),
            'backend': self.buffer(kind="nav", dtype="object"),
        }
    
    def preprocess(self):
        self.results.debug[0] = dict()
    
    def process_partition(self, partition):
        cpu = os.environ.get("LIBERTEM_USE_CPU", None)
        cuda = os.environ.get("LIBERTEM_USE_CUDA", None)
        self.results.debug[0][self.meta.slice] = (cpu, cuda)
        self.results.on_device[:] += self.xp.sum(partition, axis=0)
        self.results.backend[:] = self.meta.backend
        print(f"meta backend {self.meta.backend}")
        
    def merge(self, dest, src):
        de, sr = dest['debug'][0], src['debug'][0]
        for key, value in sr.items():
            assert key not in de
            de[key] = value
        
        dest['on_device'][:] += src['on_device']
        dest['backend'][:] = src['backend']
            
    def get_backends(self):
        return ('cupy', 'numpy')
        

In [9]:
res = ctx.run_udf(udf=DebugUDF(), dataset=ds)
print(res['debug'].data)
print(res['on_device'].data)
print(res['backend'].data)

RuntimeError: ('Requested resources %s not available in cluster', {'compute': 1, 'unobtain': 1})

In [11]:
np.any(res['backend'].data == 'cupy')

True